# P2 MAAAI 

PREPROCESADO DE DATOS:

In [5]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar100
import numpy as np

In [6]:


# Cargar CIFAR-100
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train = x_train.astype('float32') / 255.0  # Normalizar imágenes a [0,1]
x_test = x_test.astype('float32') / 255.0

# Dividir en datos etiquetados y no etiquetados (80% sin etiquetas)
np.random.seed(42)
unlabeled_indices = np.random.choice(len(x_train), size=int(0.8 * len(x_train)), replace=False)
labeled_indices = np.setdiff1d(np.arange(len(x_train)), unlabeled_indices)

x_train_labeled = x_train[labeled_indices]
y_train_labeled = y_train[labeled_indices]
x_train_unlabeled = x_train[unlabeled_indices]

print(f"Etiquetadas: {x_train_labeled.shape}, No etiquetadas: {x_train_unlabeled.shape}")

Etiquetadas: (10000, 32, 32, 3), No etiquetadas: (40000, 32, 32, 3)


EJERCICIO 3 AUTOENCODER:

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers, Input
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.callbacks import EarlyStopping

In [7]:
# ----- Paso 1: Entrenar el Autoencoder -----

def crear_encoder():
    input_img = Input(shape=(32, 32, 3))
    
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same',
                      kernel_regularizer=regularizers.l2(0.00005))(input_img)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same',
                      kernel_regularizer=regularizers.l2(0.00005))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same',
                      kernel_regularizer=regularizers.l2(0.00005))(x)
    x = layers.BatchNormalization()(x)
    encoded = layers.MaxPooling2D((2, 2))(x)

    encoder = models.Model(input_img, encoded, name="encoder")
    return encoder

def crear_decoder(encoded):
    x = layers.UpSampling2D((2, 2))(encoded)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    
    decoded = layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)
    return decoded

def crear_autoencoder():
    input_img = Input(shape=(32, 32, 3))
    encoder = crear_encoder()
    encoded = encoder(input_img)
    decoded = crear_decoder(encoded)
    
    autoencoder = models.Model(input_img, decoded, name="autoencoder")
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder, encoder


In [ ]:
autoencoder, encoder = crear_autoencoder()
# Entrenar autoencoder solo con imágenes no etiquetadas

early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

autoencoder.fit(x_train_unlabeled, x_train_unlabeled, epochs=50, batch_size=128, validation_split=0.1, callbacks=[early_stopping])


Epoch 1/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 118s 395ms/step - loss: 0.0268 - val_loss: 0.0370
Epoch 2/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 108s 382ms/step - loss: 0.0113 - val_loss: 0.0143
Epoch 3/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 98s 346ms/step - loss: 0.0082 - val_loss: 0.0082
Epoch 4/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 97s 346ms/step - loss: 0.0071 - val_loss: 0.0070
Epoch 5/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 98s 347ms/step - loss: 0.0064 - val_loss: 0.0065
Epoch 6/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 98s 346ms/step - loss: 0.0061 - val_loss: 0.0064
Epoch 7/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 99s 350ms/step - loss: 0.0055 - val_loss: 0.0062
Epoch 8/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 98s 346ms/step - loss: 0.0056 - val_loss: 0.0060
Epoch 9/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 107s 379ms/step - loss: 0.0054 - val_loss: 0.0068
Epoch 10/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 116s 410ms/step - loss: 0.0050 - val_loss: 0.0057
Epoch 11/50
 43/282 ━━━━━━━━━━━━━━━━━━━━ 1:31 384ms/step - loss: 0.0050

In [ ]:
# ----- Paso 2: Entrenar el Clasificador -----

def crear_clasificador(encoder):
    encoder.trainable = False  # Congelamos el encoder preentrenado
    input_img = encoder.input
    x = encoder.output
    x = layers.Flatten()(x)
    
    x = layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.6)(x)
    
    output = layers.Dense(100, activation='softmax')(x)
    
    classifier = models.Model(input_img, output, name="clasificador")
    classifier.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                       loss='sparse_categorical_crossentropy',
                       metrics=['accuracy'])
    return classifier


In [ ]:

classifier = crear_clasificador(encoder)

# Definir Early Stopping para detener si no mejora la validación en 5 épocas seguidas
early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

# Entrenar solo con datos etiquetados
classifier.fit(x_train_labeled, y_train_labeled, epochs=50, batch_size=128, validation_split=0.1, callbacks=[early_stopping])


NameError: name 'encoder' is not defined